In [16]:
import psycopg2.pool

import psycopg2

import yaml

In [17]:
with open('../../keys.yml', 'r') as file:
    keys = yaml.safe_load(file)

aws_keys = keys['Chimere_Aws_databse']

In [27]:
with open('../../database_structure.yml', 'r') as file:
    db_struct = yaml.safe_load(file)

In [18]:


pool = psycopg2.pool.SimpleConnectionPool(1, 20,
                                            host=aws_keys["host"],
                                            port=aws_keys["port"],
                                            database=aws_keys["database"],
                                            user=aws_keys["user"],
                                            password=aws_keys["password"])



# Testing sql queries

In [9]:
query = "insert into USER_DETAILS (Phone_Number, Name) values ('+919958498945','Test1')"


In [12]:
query = "UPDATE USER_DETAILS SET Name = 'test1', Email = 'test1@gmail.com' where Phone_Number = '+919958498945'"


In [3]:
columns = ["phone","Address"]
#

In [12]:
query = "select phone, Address from chimera_user "
query = "insert into chimera_user (phone, Address) values ('+919958498945','ABC Address')"
query = "select phone, Address from chimera_user where phone like '+919958498945'"

In [13]:
conn = pool.getconn()
conn.autocommit = True
with conn.cursor() as cursor:
    cursor.execute(query)
    # record = cursor.fetchall()
pool.putconn(conn)
# record


# Making Functions

In [24]:
def execute_enter_data( table_name:str,
                        input_dict:dict, 
                        # column_names:list,
                        # values:list,
                        update_table:bool = False,
                        where_dict:dict = None):
                        
    """The `execute_enter_data` function is used to either insert new data or update existing data in a PostgreSQL database table. The function takes in the following arguments:

        - `table_name` : a string representing the name of the table where data is to be inserted/updated.
        - `input_dict` : a dictionary containing the column names as keys and the values to be inserted/updated as values.
        - `update_table` : a boolean indicating whether data is to be updated in the table or not. Default is False.
        - `where_dict` : a dictionary containing the column names and values to be used in the WHERE clause for updating data.

        The function returns `1` if the query was executed successfully, `-1` otherwise.
        
        Example usage for inserting data:

        table_name = 'students'
        input_dict = {'name': 'John', 'age': 25, 'gender': 'M'}
        execute_enter_data(table_name, input_dict)

        Example usage for updating data:

        table_name = 'students'
        input_dict = {'name': 'John Doe', 'age': 26, 'gender': 'M'}
        where_dict = {'name': 'John', 'age': 25}
        execute_enter_data(table_name, input_dict, update_table=True, where_dict=where_dict)
    """
    if update_table:

        set_query = " , ".join(map(lambda x: rf"{x} = {input_dict[x]}",input_dict.keys()))

        query = f"UPDATE {table_name} SET {set_query}"

        if where_dict!=None:
            where_clause    = " AND ".join(map(lambda x : f"{x} like '{where_dict[x]}'",where_dict))
            query           = query + " where "+where_clause
        
        
    else:
    
        columns_to_enter = ",".join(map(str,input_dict.keys()))
        values_to_enter  = ",".join(map(lambda x: rf"{x}",input_dict.values()))    
        
        # columns_to_enter = ",".join(map(str,column_names))
        # values_to_enter  = ",".join(map(lambda x: rf"'{x}'",values))    
        
        #base query                        
        query = f"insert into {table_name} ({columns_to_enter}) values ({values_to_enter})"


    # return query
    try:
        # Get a connection from the pool
        conn = pool.getconn()
        conn.autocommit = True
        with conn.cursor() as cursor:
            cursor.execute(query)
            return 1
            # record = await cursor.fetchone()
            # return jsonify({"id": record[0], "name": record[1], "cost": record[2]})

    except (Exception, psycopg2.Error) as error :
        print(query)
        print ("Error while inserting data from PostgreSQL -->", error)
        return -1

    finally:
        # Return the connection to the pool
        if(conn):
            pool.putconn(conn)
            print("PostgreSQL connection returned to the pool")

In [ ]:
execute_enter_data()

In [21]:
def execute_get_data(table_name:str, 
                    column_names:dict = None, 
                    where_dict:dict = None):

    if column_names != None:
        columns_to_return = ",".join(map(str,column_names))
    else:
        columns_to_return = "*"
    #base query                        
    query = f"select {columns_to_return} from {table_name}"

    if where_dict!=None:
        where_clause    = " AND ".join(map(lambda x : f"{x} like '{where_dict[x]}'",where_dict))
        query           = query + " where "+where_clause
    # return query

    try:
        # Get a connection from the pool
        conn = pool.getconn()
        conn.autocommit = True
        with conn.cursor() as cursor:
            cursor.execute(query)
            record = cursor.fetchall()
            return record

    except (Exception, psycopg2.Error) as error :
        print(query)
        print ("Error while fetching data from PostgreSQL -->", error)
        return -1

    finally:
        # Return the connection to the pool
        if(conn):
            pool.putconn(conn)
            print("PostgreSQL connection returned to the pool")

# Testing Functions

In [26]:

execute_get_data(table_name="USER_BALANCE")

In [31]:
list(db_struct['USER_BALANCE'].keys())

['Phone_Number', 'BALANCE', 'CURRENCY']

In [22]:
execute_get_data("USER_DETAILS", ["*"])

PostgreSQL connection returned to the pool


[('+919958498945',
  'test1',
  'test1@gmail.com',
  None,
  None,
  None,
  None,
  None,
  None)]

In [13]:
execute_enter_data("chimera_user", ["phone","Address"], ["+919958498988","0x7281F1C206e60ad12011407a60444a721A2f4D76"])

PostgreSQL connection returned to the pool


1

In [21]:
result = execute_get_data("chimera_user", ["Address"], {"phone" : "+919958498955"})

PostgreSQL connection returned to the pool


In [22]:
result

[('ABC Address',)]

In [23]:
execute_get_data("chimera_user", ["phone","Address"])

PostgreSQL connection returned to the pool


[('+919958498955', 'ABC Address'),
 ('+919958498945', 'ABC Address'),
 ('+919958498965', 'ABCD'),
 ('+919958498944', '0x83b71f7ada2dff2758a7022151dfa7036469f58e'),
 ('+919999999997', '0x83b71f7ada2dff2758a7022151dfa7036469f58e'),
 ('+919958498988', '0x7281F1C206e60ad12011407a60444a721A2f4D76'),
 ('+919999999966', '0x83b71f7ada2dff2758a7022151dfa7036469f58e'),
 ('+919999999955', '0x83b71f7ada2dff2758a7022151dfa7036469f58e'),
 ('+919999999144', '0x83b71f7ada2dff2758a7022151dfa7036469f58e'),
 ('+919999999122', '0x83b71f7ada2dff2758a7022151dfa7036469f58e'),
 ('+919999999111', '0x83b71f7ada2dff2758a7022151dfa7036469f58e'),
 ('+919831110000', '0x83b71f7ada2dff2758a7022151dfa7036469f58e')]

In [25]:
print("Example usage for inserting data:\n\ntable_name = 'students'\ninput_dict = {'name': 'John', 'age': 25, 'gender': 'M'}\nexecute_enter_data(table_name, input_dict)\n\nExample usage for updating data:\n\ntable_name = 'students'\ninput_dict = {'name': 'John Doe', 'age': 26, 'gender': 'M'}\nwhere_dict = {'name': 'John', 'age': 25}\nexecute_enter_data(table_name, input_dict, update_table=True, where_dict=where_dict)")


Example usage for inserting data:

table_name = 'students'
input_dict = {'name': 'John', 'age': 25, 'gender': 'M'}
execute_enter_data(table_name, input_dict)

Example usage for updating data:

table_name = 'students'
input_dict = {'name': 'John Doe', 'age': 26, 'gender': 'M'}
where_dict = {'name': 'John', 'age': 25}
execute_enter_data(table_name, input_dict, update_table=True, where_dict=where_dict)


In [ ]:
async def execute_get_data(table_name, column_names =None, where_dict = None):
    
  try:
    # Get a connection from the pool
    conn = pool.getconn()

    async with conn.cursor() as cursor:
      postgreSQL_select_Query = "select * from postage where id = %s"
      await cursor.execute(postgreSQL_select_Query, (id,))
      record = await cursor.fetchone()
      return jsonify({"id": record[0], "name": record[1], "cost": record[2]})

  except (Exception, psycopg2.Error) as error :
      print ("Error while fetching data from PostgreSQL", error)

  finally:
      # Return the connection to the pool
      if(conn):
          pool.putconn(conn)
          print("PostgreSQL connection returned to the pool")

In [ ]:
async def execute_query(query):
  try:
    # Get a connection from the pool
    conn = pool.getconn()

    async with conn.cursor() as cursor:
      postgreSQL_select_Query = "select * from postage where id = %s"
      await cursor.execute(postgreSQL_select_Query, (id,))
      record = await cursor.fetchone()
      return jsonify({"id": record[0], "name": record[1], "cost": record[2]})

  except (Exception, psycopg2.Error) as error :
      print ("Error while fetching data from PostgreSQL", error)

  finally:
      # Return the connection to the pool
      if(conn):
          pool.putconn(conn)
          print("PostgreSQL connection returned to the pool")